In [1]:
# import all the libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
from string import digits
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
import nltk
# nltk.download()
import re 
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer 
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vicky\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vicky\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vicky\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#   read data from CSV file
df = pd.read_csv('data.csv',encoding='latin1')
data = df.copy()

In [3]:
data.head(10)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,...,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,...,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,...,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,...,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,...,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,...,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN
5,815719231,False,finalized,3,10/27/15 1:47,female,1.0000,yes,1.0,3/11/10 18:14,...,https://pbs.twimg.com/profile_images/656336865...,0,0,"Ive seen people on the train with lamps, chair...",NaN,20036,10/26/15 12:40,6.587300e+17,New York Gritty,Central Time (US & Canada)
6,815719232,False,finalized,3,10/27/15 1:57,brand,1.0000,yes,1.0,4/24/08 13:03,...,https://pbs.twimg.com/profile_images/528547133...,0,0,@BpackEngineer Thank you for your patience whi...,NaN,13354,10/26/15 12:40,6.587300e+17,Worldwide,Eastern Time (US & Canada)
7,815719233,False,finalized,3,10/26/15 23:48,male,1.0000,yes,1.0,12/3/12 21:54,...,https://pbs.twimg.com/profile_images/508875440...,0,C0DEED,Gala Bingo clubs bought for å£241m: The UK's l...,NaN,112117,10/26/15 12:40,6.587300e+17,NaN,NaN
8,815719234,False,finalized,3,10/27/15 1:52,female,1.0000,yes,1.0,9/8/15 4:50,...,https://pbs.twimg.com/profile_images/658670112...,0,0,@_Aphmau_ the pic defines all mcd fangirls/fan...,NaN,482,10/26/15 12:40,6.587300e+17,NaN,NaN
9,815719235,False,finalized,3,10/27/15 1:49,female,1.0000,yes,1.0,5/13/11 3:32,...,https://pbs.twimg.com/profile_images/513327289...,0,FFFFFF,@Evielady just how lovely is the tree this yea...,NaN,26085,10/26/15 12:40,6.587300e+17,"Nottingham, England.",Amsterdam


In [4]:
# see data shape 
data.shape

(20050, 26)

In [5]:
# see data info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20050 entries, 0 to 20049
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _unit_id               20050 non-null  int64  
 1   _golden                20050 non-null  bool   
 2   _unit_state            20050 non-null  object 
 3   _trusted_judgments     20050 non-null  int64  
 4   _last_judgment_at      20000 non-null  object 
 5   gender                 19953 non-null  object 
 6   gender:confidence      20024 non-null  float64
 7   profile_yn             20050 non-null  object 
 8   profile_yn:confidence  20050 non-null  float64
 9   created                20050 non-null  object 
 10  description            16306 non-null  object 
 11  fav_number             20050 non-null  int64  
 12  gender_gold            50 non-null     object 
 13  link_color             20050 non-null  object 
 14  name                   20050 non-null  object 
 15  pr

In [6]:
#  drop unnneccessary colummn that we aint gonna use in this project
data.drop(['_unit_id','_last_judgment_at','created','fav_number','profileimage','retweet_count','tweet_coord',
         '_trusted_judgments', 'tweet_count', 'tweet_created', 'tweet_id', 'tweet_location', 'user_timezone', 
         '_golden','_unit_state', 'gender_gold', 'link_color', 'name', 'profile_yn_gold', 'sidebar_color', 
         'profile_yn', 'profile_yn:confidence','gender:confidence'], axis=1, inplace=True)

In [7]:
# check the new data again
data.head(10)

,gender,description,text
0,male,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...
1,male,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...
2,male,louis whining and squealing and all,i absolutely adore when louis starts the songs...
3,male,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...
4,female,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...
5,female,you don't know me.,"Ive seen people on the train with lamps, chair..."
6,brand,"A global marketplace for images, videos and mu...",@BpackEngineer Thank you for your patience whi...
7,male,The secret of getting ahead is getting started.,Gala Bingo clubs bought for å£241m: The UK's l...
8,female,Pll Fan // Crazy about MCD // Ramen is bae,@_Aphmau_ the pic defines all mcd fangirls/fan...
9,female,"Renaissance art historian, University of Notti...",@Evielady just how lovely is the tree this yea...


In [8]:
# clean data,first we will check the sum of null data and then drop the null value
data.isna().sum()

gender           97
description    3744
text              0
dtype: int64

In [9]:
# drop the null value
data.dropna(axis=0,inplace=True)

In [10]:
# see data info after we dropped the null values,we will seee that the number of Non-Null count has changed 
# as we dropped the null values 
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16224 entries, 0 to 20049
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   gender       16224 non-null  object
 1   description  16224 non-null  object
 2   text         16224 non-null  object
dtypes: object(3)
memory usage: 507.0+ KB


In [11]:
# next step,we want to check the gender in gender column, in this project we only care for females and males
data['gender'].value_counts()

female     5725
male       5469
brand      4328
unknown     702
Name: gender, dtype: int64

In [12]:
# as we only care about females and males so we will save only this two value for gender column 
data['gender'] = data[(data['gender'] == 'female') | (data['gender'] == 'male')]

In [13]:
# now gender coulumn only contains two values: female and male.
data['gender'].value_counts()

female    5725
male      5469
Name: gender, dtype: int64

In [14]:
# encode gender column for futher analysis 
for gender in df['gender']:
  if gender=='male':
     data['gender'].replace({'male':'1'},inplace=True)
  elif gender=='female':
     data['gender'].replace({'female':'0'},inplace=True)
# check the data again 
data['gender'].value_counts()

0    5725
1    5469
Name: gender, dtype: int64

In [15]:
# now we will see that our data isnt quite clean yet so the nest step we will clean the description and text column
data.head(100)

,gender,description,text
0,1,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...
1,1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...
2,1,louis whining and squealing and all,i absolutely adore when louis starts the songs...
3,1,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...
4,0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...
...,...,...,...
102,NaN,Come to AGROWN's #AgTech Investor Conference a...,We got to see inside the #OARDC #greenhouses i...
103,0,Community Outreach. Medicaid Managed Care\nBoa...,Thanks to the #ACA more adults are eligible fo...
104,0,Ventas Digitales @ IBM EspaÌ±a. Todas mis opin...,"IBMSocialBiz: In the future, there will be a n..."
105,1,#Ä£FreeZa #Ä£RIPNAZERAH_ü,Chris Got On The Black Toe 1s _Ù÷_


In [16]:
# drop the null value
data.dropna(axis=0,inplace=True)

In [17]:
# Data cleaning , first we will clean,tokenize and lemmatize description clumn and later text colum will be cleaned as well

# clean decription 
def clean_decription(descrip):
    # remove the url from decription
    url_regex = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', ' ', descrip)
    # remove the punctuation from decription
    descrip_clean = re.sub(r'[^\w\s]', '', url_regex)
    # remove the number from description
    remove_digits = descrip_clean.translate(str.maketrans('', '',digits))
    # lower all the deription 
    final_descrip  = remove_digits.lower()
    
    return final_descrip

data['descrip_Cleaned'] = pd.DataFrame(data['description'].apply(lambda x: clean_decription(x))) 

# tokenize descrip
data['descrip_Cleaned'] = [nltk.word_tokenize(tweet) for tweet in data['descrip_Cleaned']]
descrip_new=[]
for each_row in data['descrip_Cleaned']:
    descrip_new.append([i for i in each_row])
descrip_alpha=[]

# remove stop word from descrip
stop_words = set(stopwords.words('english'))
for each_row in descrip_new:
    descrip_alpha.append([i for i in each_row if i not in stop_words])
    
# lemmatize descrip 
description_lemma = []
lemma = nltk.WordNetLemmatizer()
for each_row in descrip_alpha:
    description_lemma.append([lemma.lemmatize(word) for word in each_row])

data['descrip_Cleaned'] = description_lemma
data['descrip_Cleaned'] = [" ".join(desc) for desc in data['descrip_Cleaned'].values]
# check the data fot descrip
data.head()

,gender,description,text,descrip_Cleaned
0,1,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,sing rhythm
1,1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...,im author novel filled family drama romance
2,1,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining squealing
3,1,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,mobile guy er shazam google kleiner perkins ya...
4,0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson best frontmankaiser chief best ba...


In [18]:
# clean text
def clean_text(text):
     # remove the url from text
    url_regex = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', ' ', text)
    # remove the punctuation from text
    text_clean = re.sub(r'[^\w\s]', '', url_regex)
    # remove the number from text
    remove_digits = text_clean.translate(str.maketrans('', '',digits))
    # lower all the text 
    final_text  = remove_digits.lower()
    
    return final_text
data['text_Cleaned'] = pd.DataFrame(data['text'].apply(lambda x: clean_text(x)))

# tokenize text
data['text_Cleaned'] = [nltk.word_tokenize(tweet) for tweet in data['text_Cleaned']]
text_new=[]
for each_row in data['text_Cleaned']:
    text_new.append([i for i in each_row])
text_alpha=[]

# remove stop word from text
stop_words = set(stopwords.words('english'))
for each_row in text_new:
    text_alpha.append([i for i in each_row if i not in stop_words])
    
# lemmatize text 
text_lemma = []
lemma = nltk.WordNetLemmatizer()
for each_row in text_alpha:
    text_lemma.append([lemma.lemmatize(word) for word in each_row])

data['text_Cleaned'] = text_lemma
data['text_Cleaned'] = [" ".join(desc) for desc in data['text_Cleaned'].values]
# check the data fot text
data.head()

,gender,description,text,descrip_Cleaned,text_Cleaned
0,1,i sing my own rhythm.,Robbie E Responds To Critics After Win Against...,sing rhythm,robbie e responds critic win eddie edward worl...
1,1,I'm the author of novels filled with family dr...,ÛÏIt felt like they were my friends and I was...,im author novel filled family drama romance,ûïit felt like friend living story themû retir...
2,1,louis whining and squealing and all,i absolutely adore when louis starts the songs...,louis whining squealing,absolutely adore louis start song hit hard fee...
3,1,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",Hi @JordanSpieth - Looking at the url - do you...,mobile guy er shazam google kleiner perkins ya...,hi jordanspieth looking url use ifttt dont typ...
4,0,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,Watching Neighbours on Sky+ catching up with t...,ricky wilson best frontmankaiser chief best ba...,watching neighbour sky catching neighbs xxx _ù...


In [19]:
# drop the columns that we aint gonna use 
data.drop(['description','text'],axis=1,inplace=True)
# check data
data.head()

,gender,descrip_Cleaned,text_Cleaned
0,1,sing rhythm,robbie e responds critic win eddie edward worl...
1,1,im author novel filled family drama romance,ûïit felt like friend living story themû retir...
2,1,louis whining squealing,absolutely adore louis start song hit hard fee...
3,1,mobile guy er shazam google kleiner perkins ya...,hi jordanspieth looking url use ifttt dont typ...
4,0,ricky wilson best frontmankaiser chief best ba...,watching neighbour sky catching neighbs xxx _ù...


In [20]:
data[['gender','descrip_Cleaned','text_Cleaned']] = data[['gender','descrip_Cleaned','text_Cleaned']].fillna('')
# vectorize the data
cv = CountVectorizer(max_features = 1000)
x  = cv.fit_transform(data['descrip_Cleaned']).toarray()
x1 = cv.fit_transform(data['text_Cleaned']).toarray()
# store the data in data frame
A = pd.DataFrame(x)
B = pd.DataFrame(x1)
# concate the data frames together
X = pd.concat([B,A],join='outer',axis=1)
# check data shape
X.shape

(11194, 2000)

In [21]:
# check gender column shape
data['gender'].shape

(11194,)

In [22]:
# the two data frame (A,B) is store as x and our gender column is store as y 
x = np.array(X)
y = np.array(data['gender'])

In [23]:
# split data for train and test : ratio 80:20 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size= 0.2, random_state = 0)

In [24]:
print('Shape of training data: ')
print(X_train.shape)
print(y_train.shape)
print('Shape of test data: ')
print(X_test.shape)
print(y_test.shape)

Shape of training data: 
(8955, 2000)
(8955,)
Shape of test data: 
(2239, 2000)
(2239,)


In [25]:
# usinng  XGBoost Classifier to train the data
xgbmodel = XGBClassifier(max_depth= 7, min_child_weight=1)
xgbmodel.fit(X_train, y_train)
y_pred2 = xgbmodel.predict(X_test)
confusion_matrix(y_test, y_pred2)

[14:21:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


array([[876, 256],
       [543, 564]], dtype=int64)

In [26]:
print(classification_report(y_test, y_pred2))

              precision    recall  f1-score   support

           0       0.62      0.77      0.69      1132
           1       0.69      0.51      0.59      1107

    accuracy                           0.64      2239
   macro avg       0.65      0.64      0.64      2239
weighted avg       0.65      0.64      0.64      2239



In [27]:
xgb_param_grid = {
              "max_depth": [5, 10],
              "min_child_weight": [3, 4],
              }
xgbgrid = GridSearchCV(estimator = xgbmodel, param_grid = xgb_param_grid, cv = 3, n_jobs = -1, verbose = 0)

xgbgrid.fit(X_train, y_train)
xgbgrid.best_params_

[14:23:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


{'max_depth': 10, 'min_child_weight': 3}

In [28]:
y_pred2_hyper = xgbgrid.predict(X_test)
accuracy = accuracy_score(y_test, y_pred2_hyper)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 65.52%


In [29]:
confusion_matrix(y_test, y_pred2_hyper)
print(classification_report(y_test, y_pred2_hyper))

              precision    recall  f1-score   support

           0       0.64      0.74      0.69      1132
           1       0.68      0.56      0.62      1107

    accuracy                           0.66      2239
   macro avg       0.66      0.65      0.65      2239
weighted avg       0.66      0.66      0.65      2239

